In [31]:
!python -V

Python 3.10.9


In [32]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import pickle

In [5]:
import mlflow 

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='/home/danial/Desktop/mymlops/MLOPS/experiment-tracking/mlruns/1', creation_time=1684962248383, experiment_id='1', last_update_time=1684962248383, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [33]:
def read_dataframe(file_name):
    df = pd.read_parquet(file_name)
    
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds()/60)
    df = df[((df.duration >= 1) & (df.duration <= 60))]
    
    categorical = ['PULocationID', 'DOLocationID']
    
    df[categorical] =  df[categorical].astype(str)

    return df 

In [34]:
parent_directory = '/home/danial/Desktop/mymlops/MLOPS/data/' 

In [45]:
df_train = read_dataframe(parent_directory + 'green_tripdata_2021-01.parquet')
df_val = read_dataframe(parent_directory + 'green_tripdata_2021-02.parquet')
len(df_train), len(df_val)

(73908, 61921)

In [46]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [47]:
numerical = ['trip_distance']
categorical = ['PU_DO']

In [48]:
df_train[categorical + numerical]

,PU_DO,trip_distance
0,43_151,1.01
1,166_239,2.53
2,41_42,1.12
3,168_75,1.99
7,75_75,0.45
...,...,...
76513,81_90,17.63
76514,35_213,18.36
76515,74_69,2.50
76516,168_215,14.48


In [49]:
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dicts)



val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)


In [50]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


In [52]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)



7.758715202423387

In [53]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [58]:
with mlflow.start_run():

    mlflow.set_tag("developer", 'danial')

    mlflow.log_param('train-data-path', parent_directory + 'green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', parent_directory + 'green_tripdata_2021-02.parquet')

    alpha = 0.1
    mlflow.log_param('alpha', alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

In [187]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.7037351333448765

In [203]:
len(df_train['PU_DO'].unique())

13220